In [1]:
# 何回目すか,,マジで
import pandas as pd 
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random


In [2]:
df_data = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec.csv')
search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group_3_1/group_2"
file_list = list(os.listdir(search_folder))

df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
L = len(df_link)

df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

In [76]:

###### 各linkのoとdの座標を入れておく配列 ###### 
link_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0]
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    link_loc_array.append(loc_tuple) 

###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'ID']
    x_j = df_ble[df_ble['ID'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['ID'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['ID'] == beacon_id]['floor'].iloc[0]
    beacon_loc = np.array([x_j, y_j, z_j])

    beacon_loc_array.append(beacon_loc) 

###### ビーコンとリンク線分の中点間距離を返す関数 ######
def dist_to_mid(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)

    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算
    distance = np.linalg.norm(x - mid_point)
    
    return distance

###### d_arrayの用意 ###### # 各リンクの中点とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))
for i in range(len(df_link)):
    p_o = link_loc_array[i][0] # o座標 # i=1の時
    p_d = link_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = dist_to_mid(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
# 15分間のデータでやるならtimstep18secなら50step, 20secなら45step
dens_jt = np.zeros((23, 45), dtype = int)
max_jt = np.zeros((23, 45), dtype = int)
mean_jt = np.zeros((23, 45), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_data.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count

####### リンク接続行列 ####### 
def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在
    for i in range(n):
        O = link_data.loc[i, 'o'] 
        D = link_data.loc[i, 'd'] 
        for j in range(n):
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = Imaking(df_link)

for i in range(26, 45, 2): # 階段には滞在リンク入れない．向きは考えないので面倒なことは考えない
    I[i-1, i-1] = 0
    I[i, i] = 0

for i in range(46, 49): # 吸収リンクからは吸収リンクにしか接続しない
    I[i-1, :] = 0 
    I[i-1, i-1] = 1 

###### 改札の吸収リンク ######
ddata = [46, 47, 48]
D = len(ddata)

####### パラメタ&タイムステップ設定 #######
xdim = 3
x = np.zeros(xdim)
x[0] = -1
x[-1] = 0.8
sigma = 15 
T = 11 # inputデータによる．10stepで最終リンクには辿り着く人たち，吸収に行くのに+1timestep要すると考えるとT=11が妥当か．
# 20ステップ以内ならT=20;1=21

#######################
####### 観測モデル ######
#######################

####### 観測方程式 ######
def qq(dist):
    return (dist/((sigma)**2) + np.exp(-((dist)**2)/2/((sigma)**2))) # レイリー分布(?)

##### 重み付き最小二乗法 #####
def weighted_least_squares(coordinates, observations):
    # 各観測点の座標と重みを取り出す
    observed_coordinates = observations[:, :-1]  # 最後の列（重み）を除いた座標
    weights = observations[:, -1]  # 重み  
    errors = coordinates - observed_coordinates
    weighted_errors = weights * np.sum(errors ** 2, axis=1)
    
    return np.sum(weighted_errors)

# 観測モデル1の準備
saikyo_key = [1, 2, 3, 4, 5, 6]
yamate_key = [7, 8, 9, 10, 11, 12, 13]
discriminate_key = [19, 22, 23]
abs_beacons = [16, 20, 21]

# ３点測量による位置推定
def measuring_model(x):
    print(f'パラメタ{x}で観測モデル回す')
    measured_loc = np.zeros(6) # 'userid', 'timestep', 'x', 'y', 'z' 

    for file_name in file_list:
        file_path = os.path.join(search_folder, file_name)
        dfi = pd.read_csv(file_path)
        base_name = file_name.split('.')[0]   
        userid = int(base_name)

        nts = dfi['timestep'].nunique()                
        grouped = dfi.groupby('timestep')
        dfi_list = [group.reset_index(drop=True) for name, group in grouped]  
        i_res = np.zeros(6)
            
        for t in range(nts): 
            dfit = dfi_list[t]
            ts = int(dfit.loc[0, 'timestep'])

            result_beacon = dfit.groupby('ID')
            result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]
            tdict = {}

            for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                dfitj = result_beacon_list[j]
                jscore = 0 
                jid = dfitj.loc[0, 'ID'] 
                max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi

                for k in range(len(dfitj)): 
                    rssi = dfitj.loc[k, 'RSSI']                        
                    jscore += max_rssi / rssi 
                    tdict[jid] = jscore  
            
            jmax = max(tdict, key = tdict.get) # 最大ビーコンのidをメモ
            
            if t < nts-1: # t=が最後の時は次の最大ビーコンの座標を参照できないので場合わけ
                if len(tdict) > 1: # 2点以上で観測されてたら，最小二乗法で一点の推定位置を出すことが可能  
                    observation_array = np.zeros(4)
                    for b in range(len(tdict)):
                        beacon_i = list(tdict.keys())[b] # beacon_iはビーコンのidなので-1してindexにする
                        beacon_i = int(beacon_i)
                        loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                        beacon_w = list(tdict.values())[b]

                        result_array = np.insert(loc_i, len(loc_i), beacon_w)
                        observation_array = np.vstack((observation_array, result_array))
                    
                    # 先頭の0の行を削除
                    observation_array = np.delete(observation_array, 0, axis = 0)

                    # 初期座標の設定（仮の値を使用，ある程度それっぽい値を入れないと変な値を返してくるので注意）
                    initial_coordinates = np.array([-11920, -37880, 20])

                    # 重み付き最小二乗法を最小化して推定座標を求める
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    # 推定された座標を取得
                    estimated_coordinates = result.x
                    estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts) # tsを列の先頭に追加

                elif len(tdict) == 1: # その時点tが最後(nts-1)ではなく，かつ観測が1ビーコンだったら，次の観測を参照
                    observation_array = np.zeros(4)
                    beacon_i = list(tdict.keys())[0] # beacon_iはビーコンのidなので-1してindexにする
                    beacon_i = int(beacon_i)
                    loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                    beacon_w = list(tdict.values())[0]

                    result_array = np.append(loc_i, float(beacon_w))
                    observation_array = np.vstack((observation_array, result_array))

                    # 次のmaxbeaconを参照
                    dfitt = dfi_list[t+1]     
                    next_result_beacon = dfitt.groupby('ID')
                    next_result_beacon_list = [group.reset_index(drop=True) for name, group in next_result_beacon]

                    ttdict = {} # 時刻t+1の各ビーコンのscoreを記録するdictの準備

                    for j in range(len(next_result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                        dfittj = next_result_beacon_list[j]
                        next_jscore = 0 
                        next_jid = dfittj.loc[0, 'ID'] 
                        next_max_rssi = max_jt[next_jid-1, (ts+1)-1] # tsでのbeaconのmaxのrssi

                        for k in range(len(dfittj)): # freq = count / count_tot
                            next_rssi = dfittj.loc[k, 'RSSI']                        
                            next_jscore += next_max_rssi / next_rssi # beacon_eval(rssi, max_rssi) # , freq)
                            ttdict[next_jid] = next_jscore

                    next_max_key = max(ttdict, key = ttdict.get)
                    next_max_score = ttdict[next_max_key]

                    next_max_beacon_loc = beacon_loc_array[next_max_key-1] # 次の最大ビーコンの座標

                    next_result_array = np.insert(next_max_beacon_loc, len(next_max_beacon_loc), next_max_score)
                    observation_array = np.vstack((observation_array, next_result_array))

                    observation_array = np.delete(observation_array, 0, axis = 0)

                    initial_coordinates = np.array([-11920, -37880, 20])
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    estimated_coordinates = result.x
                    estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts) # tsを列の先頭に追加(ts, x, y, z)
            
            elif t == nts-1: # 最後は一番検出が顕著だったビーコンの座標で          
                last_beacon = max(tdict, key = tdict.get)
                last_beacon = int(last_beacon)
                estimated_coordinates = beacon_loc_array[last_beacon-1] 
                estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts)

            estimated_coordinates_with_ts_i = np.insert(estimated_coordinates_with_ts, 0, userid) # useridを列の先頭に追加
            i_res = np.vstack((i_res, estimated_coordinates_with_ts_i))

        measured_loc = np.vstack((measured_loc, i_res))
        measured_loc_df = pd.DataFrame(measured_loc, columns = ['userid', 'timestep', 'maxbeacon', 'x', 'y', 'z'])

    return measured_loc_df

###### 観測モデル2 ######
# モデル1の結果から経路選択モデルの情報使わずに実施．個人ごとにt, k, aの列(要はdfi_data)を作成する
def measuring_model2(x):

    # モデル1を回す
    measured_loc_df = measuring_model(x)
    grouped = measured_loc_df.groupby('userid')
    df_list = [group for name, group in grouped]
    df_list = [data.reset_index(drop=True) for data in df_list]   

    # アウトプットの準備
    columns = ['userid', 'timestep', 'k', 'a']
    result_df = pd.DataFrame(columns = columns)
    
    # 以降個人ごとに処理（OD決定→個人プリズムの作成&経路選択モデル用のプリズムの作成→マッチング→経路データの作成）
    for i in range(len(df_list)):
        dfi = df_list[i]
        abs = dfi.loc[len(dfi)-1, 'maxbeacon']
        userid = int(dfi.loc[0, 'userid'])
        maxbeacon_list = dfi['maxbeacon'].tolist()
        Ti = len(dfi) # 個人の観測限界

        ###### 個人ごとにdfi_dataを入れるのでよくね ######
        dfi_data = pd.DataFrame(columns = columns)
        # useridの値をdfi_dataの'userid'列に代入
        dfi_data['userid'] = [userid] * T

        # dfi_dataの行数をTに設定
        dfi_data = dfi_data.head(T)

        # 吸収がハチ公・中央以外だったら飛ばす
        if abs not in abs_beacons:
            continue
        
        # O決める
        first_beacon = int(dfi.loc[0, 'maxbeacon'])
        first_dist_list = d_array[:, first_beacon-1] 
        first_dist_dict = {index + 1: value for index, value in enumerate(first_dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

        if any(item in discriminate_key for item in maxbeacon_list): # 埼京オンリーにする
            filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in saikyo_key}
            max_key = min(filtered_dict, key = filtered_dict.get) # これが初期リンクとなる．埼京ホームのうち，最もmax_beaconに近いリンク
        else: 
            filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in yamate_key}
            max_key = min(filtered_dict, key = filtered_dict.get)
                  
        a0 = int(max_key) 

        # 次にDを確定
        last_beacon = int(dfi.loc[Ti-1, 'maxbeacon']) # 一番下の行を指定
        last_link = 0
        if last_beacon == 16: # node39
            last_link = 25 
            abs_link = 46
        elif last_beacon == 20: # node28
            last_link = 17
            abs_link = 47
        elif last_beacon == 21: # node29
            last_link = 18   
            abs_link = 48   
        else:
            continue
        
        # ODの確定
        ao = a0 - 1 
        ad = abs_link -1 
        atk = a0 # 初期値

        # OD決まったら時空間プリズム制約Idを入れる（観測モデル内ではIdmのみ，Idrlは不要）
        Idm = np.zeros((Ti, L, L)) # t=Ti-2でlast linkに到達するので．t=Ti-1でabs linkに到達するようにする
        Itmlist = np.zeros((Ti+1, L, L))
        IIm = np.copy(I)
        Itmlist[0, :, :] = np.eye(L) 

        for tt in range(1, Ti+1): 
            Itmlist[tt, :, :] = IIm 
            IIm = np.dot(IIm, I) 
            IIm = np.where(IIm > 0, 1, 0) 

        Ittmlist = np.zeros((Ti+1, L, L))
        for tt in range(1, Ti+2): 
            Ittmlist[tt-1, :, :] = np.transpose(Itmlist[(Ti+1) - tt, :, :]) 

        for tt in range(Ti):
            if tt == 0:
                Idm[tt, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね，と思うが．
                continue

            alist = np.where(Itmlist[tt + 1, ao, :] == Ittmlist[tt + 1, ad, :])[0]  
            for a in alist:
                if Itmlist[tt + 1, ao, a] == Ittmlist[tt + 1, ad, a]:
                    klist = np.where(I[:, a] == 1)[0] 

                    for k in klist:
                        if len(np.where(Idm[tt - 1, :, k] == 1)[0]) != 0:
                            Idm[tt, k, a] = 1 

        # 時刻tでのプリズム条件と観測結果から各時刻のk, aを決めていく
        for t in range(T): # 列はidとt, k, a．ata = at+1_k
            if (t <= (Ti-2)):
                Idmt = Idm[t+1, :, :] # 時刻tにおいて次に利用可能なリンク（＝aの候補）#### t+1にしたら上手くいってるけど，，ちょっと謎

                dfi_data.loc[t, 'k'] = atk # tの時にいたリンクはatk
                a_row = Idmt[atk-1, :]
                ata_cand = np.where(a_row == 1)[0]+1

                # 遷移「先」の位置を特定しようとしているのでt+1にする
                estx = dfi.loc[t+1, 'x']
                esty = dfi.loc[t+1, 'y']
                estz = dfi.loc[t+1, 'z']
                estloc = np.array([estx, esty, estz])

                res_dict = {} 

                if len(ata_cand) == 0:
                    break

                for ata in ata_cand: # 次リンクataの中点位置(po+pd/2)とt+1でのestimated_locから計算する
                    p_o = link_loc_array[ata-1][0]
                    p_d = link_loc_array[ata-1][0]
                    dist = dist_to_mid(p_o, p_d, estloc) 

                    q = qq(dist) # ataの観測尤度
                    res_dict[ata] = q # 事前情報による補正なし（プリズムという強い事前情報はある）

                if not res_dict:
                    print('sss')
                    break

                else:
                    max_delta_key = max(res_dict, key = res_dict.get)
                    dfi_data.loc[t, 'a'] = max_delta_key    # これが次のdfi_data.loc[t, 'k']になると言うことじゃ！
                    atk = max_delta_key                     # 次のatkなので

            else: 
                dfi_data.loc[t, 'k'] = abs_link
                dfi_data.loc[t, 'a'] = abs_link   
            
            dfi_data.loc[t, 'timestep'] = t
        
        if dfi_data.isnull().values.any():
            continue
        else:
            result_df = pd.concat([result_df, dfi_data], axis = 0, ignore_index = True)

    return result_df             


In [100]:
##########################
###### 経路選択モデル ######
##########################

###### 経路選択モデリングのための時空間プリズム ######
# 起点Oからの到達可能性指示変数ioの定義 kから出発して時刻（遷移回数）t以内に到達できるリンクaがII[t, k, a]=1になっている
Itlist = np.zeros((T+1, L, L))
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) # ItlistにはT+1成分あるが，その第一成分(t=0)は単位行列ということ．t=1以降はfor内で計算

for ts in range(1, T+1): 
    Itlist[ts, :, :] = II 
    II = np.dot(II, I)
    II = np.where(II > 0, 1, 0) 

# 終点Dからの到達可能性指示変数idの定義
Ittlist = np.zeros((T+1, L, L))
for ts in range(1, T+2): 
    Ittlist[ts-1, :, :] = np.transpose(Itlist[(T+1) - ts, :, :]) 

###### 即時効用行列 ######
"""
def Mset(x): 
    cost = np.exp(df_link['length']/10 * x[0]) # + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
    cost = pd.concat([cost]*L, axis=1)
    cost = cost.T
    cost_numpy = cost.values # DataFrameをNumPy配列に変換
    return cost_numpy
"""
###### 即時効用行列 ###### # 混雑項を入れるなら時間毎で変化するので今はL*Lの二次元配列だが，これにtの次元を加えてT*L*Lの3次元配列に変更
def Mset(x): 
    #Probs_T = Probs.T # Probsを読み込む
    inst = np.zeros((T, L, L))
    for t in range(T):
        inst_t = np.exp(df_link['length']/10 * x[0]) # + Probs_T[:, t] * x[1])    # + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
        inst_t = pd.concat([inst_t]*L, axis=1) 
        inst_t = inst_t.T
        inst_t_numpy = inst_t.values # DataFrameをNumPy配列に変換
        inst[t, :, :] = inst_t_numpy

    return inst

###### 対数尤度関数 ######
def loglikelihood(x):
    beta = x[-1]
    LL = 0
    
    # 観測モデル回し結果をuseridで分割
    result_df = measuring_model2(x) 
    grouped = result_df.groupby('userid')
    df_list = [group for name, group in grouped]
    df_list = [data.reset_index(drop=True) for data in df_list]    
    
    #### 以降個人ごとに処理 ####（経路データからOD読み込み→プリズム作成→尤度関数計算）
    for i in range(len(df_list)):
        dfi = df_list[i]
        Ti = len(dfi) # 個人の観測限界
        # 知りたいのは各時刻のk, aと全体のoとd
        a0 = dfi.loc[0, 'k']
        abs_link = dfi.loc[Ti-1, 'a']
        
        # OD確定
        ao = a0 - 1 
        ad = abs_link -1 

        # OD→プリズム制約Id
        Idrl = np.zeros((T, L, L)) 

        for ts in range(T):
            if ts == 0:
                Idrl[ts, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね，と思うが．
                continue

            alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, ad, :])[0] 
            for a in alist:
                if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, ad, a]: # これelseなわけない（当然）
                    klist = np.where(I[:, a] == 1)[0]

                    for k in klist:
                        if len(np.where(Idrl[ts - 1, :, k] == 1)[0]) != 0:
                            Idrl[ts, k, a] = 1 

        ##### 即時効用 #####
        M = np.zeros((T, L, L))
        for ts in range(T):
            Mts = Idrl[ts, :, :] * Mset(x)[ts, :, :] # ここにプリズム制約が入るのでODを考慮できている
            M[ts, :, :] = Mts
        
        #print(f'')

        ##### 価値関数 ##### 
        z = np.ones((T+1, L))
        for t in range(T, 0, -1):
            zii = M[t-1, :, :] * (z[t, :] ** beta) 
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
        
        klink = list(dfi.loc[:, 'k'])
        alink = list(dfi.loc[:, 'a'])        

        ##### 対数尤度 #####
        for t in range(T): 
            k = int(klink[t]) - 1 # at
            a = int(alink[t]) - 1 # at+1
     
            if M[t, k, a] == 0: # 接続条件を満たせなかった観測は排除
                break

            LL += np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])

    print(f'パラメタ{x}で回して尤度は{LL}')
    return -LL


In [102]:
x1 = np.array([-1.3, -0.1, 0.8])
np.set_printoptions(threshold=np.inf)
print(Mset(x1).shape)

(11, 48, 48)


In [95]:
x1 = np.array([-1.3, -0.1, 0.8])
np.set_printoptions(threshold=np.inf)

print(loglikelihood(x1))

パラメタ[-1.3 -0.1  0.8]で観測モデル回す


/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_22222/1109974856.py:44: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算


パラメタ[-1.3 -0.1  0.8]で回して尤度は-163.58649113713417
163.58649113713417


尤度関数の方はちゃんと動いているみたいだけどnewPの調子が悪い
→内生性に関しては解消済み！

In [113]:
###### リンク選択確率計算（混雑内生化） ###### ### 目的はProbsを返すこと！！
## これは観測モデルとは関係ない．dfi_dataを所与として読み込み，あとは経路選択モデルだけ考えている
def newP(x):
    Probs = np.zeros((T, L)) ## 時間tでのリンクkの期待人数＝個人ごとの選択確率の総和
    beta = x[-1]

    # 観測モデル回し結果をuseridで分割
    result_df = measuring_model2(x) 
    grouped = result_df.groupby('userid')
    df_list = [group for name, group in grouped]
    df_list = [data.reset_index(drop=True) for data in df_list]    
    
    #### 以降個人ごとに処理 ####（経路データからOD読み込み→プリズム作成→尤度関数計算）
    for i in range(len(df_list)):
        dfi = df_list[i]
        Ti = len(dfi) # 個人の観測限界
        a0 = dfi.loc[0, 'k']
        abs_link = dfi.loc[Ti-1, 'a']
        userid = int(dfi.loc[0, 'userid'])

        # OD確定
        ao = a0 - 1 
        ad = abs_link -1 

        # OD→プリズム制約Id
        Idrl = np.zeros((T, L, L))

        for ts in range(T):
            if ts == 0:
                Idrl[ts, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね？？
                continue

            alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, ad, :])[0] 
            for a in alist:
                if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, ad, a]: # always True
                    klist = np.where(I[:, a] == 1)[0]

                    for k in klist:
                        if len(np.where(Idrl[ts - 1, :, k] == 1)[0]) != 0:
                            Idrl[ts, k, a] = 1 

        ##### 即時効用 #####
        M = np.zeros((T, L, L))
        for ts in range(T):
            Mts = Idrl[ts, :, :] * Mset(x)[ts, :, :] # *は要素毎の積
            non_zero_indices = np.nonzero(Mts)
            #print(f'個人{i}時刻{ts}のMtsの非零成分の位置は{non_zero_indices}') # 最後の吸収リンク以外機能してない
            #print(f'個人{i}時刻{ts}のMtsは{Mts}')
            M[ts, :, :] = Mts

        #print(f'個人{userid}のMは{M}') # 多分Mはちゃんとできている．なのになんで動いてくれないの

        ##### 価値関数 ##### 
        z = np.ones((T+1, L))
        for t in range(T, 0, -1):
            zii = M[t-1, :, :] * (z[t, :] ** beta) 
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
        
        ##### 選択確率行列 ##### # 尤度計算は不要
        Pi = np.zeros((T, L, L)) # 初期設定
        for t in range(T): # 選択確率は各時刻tごとに決まる
            for k in range(L):
                for a in range(L):
                    if M[t, k, a] == 0: # 接続条件を満たせなかった観測は排除
                        #### なぜか全時刻に対して1, 2, 3, 26しか1になってない！！！！他が全部0になってる
                        continue # ここがbreakになってたのが癌だった説濃厚

                    Pi[t, k, a] += np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])) 
                    # Piが明らかにおかしい
                    #print(f'個人{i}の時刻{t}の{k+1}から{a+1}への遷移の確率は{Pi[t, k, a]}')
                    # それかMt, ztを入れてやるか．そもそもMtとかZtを時間別で考えているのは時空間プリズムを考えているからであり，それがなければMやZは個人に対して時不変で良い（はず）
            # print(f'個人{userid}時刻{ts}のPitは{Pi[t, :, :]}') # Piが明らかにおかしい現象は解消したと思う
        # ここまでで個人iの時刻別の遷移確率行列Piが出揃ったのでLCP(link choice probability)を計算（個人ごと）
        LCP = np.zeros((T, L))
        LCP[0, ao] += 1 # 最初t=0はa0に居るので，index aoのところが1になる．それ以外は0のまま
        #print(f'個人{i}時刻{0}のLCPは{LCP[0, :]}です')
        for t in range(1, T):
            LCP[t, :] = np.dot(LCP[t-1, :], Pi[t, :, :]) # *は要素ごとの積→行列積は@の方がいい？？？
            #if LCP[t, :] == 0:
                # これはこの人は時刻tにどこにもいないことになる
                
            #    continue
            # print(f'個人{userid}時刻{t}のLCPは{LCP[t, :]}です') # LCPはリンク滞在確率なので，各時点にそのリンクにいる確率ね．
            # print(LCP[t, :]) # これが全部0になってる可能性大 # 解消されたと思う
        
            # 個人毎に足していく
            Probs[t, :] += LCP[t, :]
    
    return Probs

In [114]:
x1 = np.array([-1, -0.1, 0.8])
np.set_printoptions(threshold=np.inf)

Probs0 = newP(x1)
print(Probs0)

パラメタ[-1.  -0.1  0.8]で観測モデル回す


/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_22222/1109974856.py:44: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算


[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.97111948e+00 3.71349359e+00 5.18367542e+00 2.52101376e+00
  9.36695166e-01 0.00000000e+00 7.69191823e-01 3.31672049e+00
  2.05588482e+00 4.11977588e+00 3.36013207e+00 7.22854736e+00
  1.79274075e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.000

In [52]:
print(Probs0[1]) # t=0の時の各リンクの選択確率

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
